In [1]:
import numpy as np
import re

Преобразуем вход в двоичные слова

In [2]:
data = [168, 42, 21, 98, 32, 17, 249, 36, 8, 48, 179, 216, 91, 42, 50, 185, 220, 51, 176, 16, 168, 5, 81, 53, 152, 76, 228, 81, 181, 92, 224, 3, 240, 9, 54]

In [3]:
message = np.array(data)

In [4]:
len(message)

35

In [5]:
message

array([168,  42,  21,  98,  32,  17, 249,  36,   8,  48, 179, 216,  91,
        42,  50, 185, 220,  51, 176,  16, 168,   5,  81,  53, 152,  76,
       228,  81, 181,  92, 224,   3, 240,   9,  54])

In [6]:
message_bin = np.vectorize(np.binary_repr)(message, width=8)

In [7]:
message_bin

array(['10101000', '00101010', '00010101', '01100010', '00100000',
       '00010001', '11111001', '00100100', '00001000', '00110000',
       '10110011', '11011000', '01011011', '00101010', '00110010',
       '10111001', '11011100', '00110011', '10110000', '00010000',
       '10101000', '00000101', '01010001', '00110101', '10011000',
       '01001100', '11100100', '01010001', '10110101', '01011100',
       '11100000', '00000011', '11110000', '00001001', '00110110'],
      dtype='<U8')

In [8]:
message_str = ''.join(message_bin)

In [9]:
message_str

'1010100000101010000101010110001000100000000100011111100100100100000010000011000010110011110110000101101100101010001100101011100111011100001100111011000000010000101010000000010101010001001101011001100001001100111001000101000110110101010111001110000000000011111100000000100100110110'

Необходимо добавить нулей в конец, чтобы мы могли поделить на кодовые слова.

In [10]:
n = 35

In [11]:
size = n - int(np.log2(n) + 1)

In [12]:
size

29

In [13]:
shift = n - len(message_str) % n

In [14]:
shift

35

In [15]:
message_str = message_str + '0' * shift

In [16]:
words = re.findall('.{%s}' % n, message_str)

Полученный набор кодовых слов

In [17]:
words

['10101000001010100001010101100010001',
 '00000000100011111100100100100000010',
 '00001100001011001111011000010110110',
 '01010100011001010111001110111000011',
 '00111011000000010000101010000000010',
 '10101000100110101100110000100110011',
 '10010001010001101101010101110011100',
 '00000000011111100000000100100110110',
 '00000000000000000000000000000000000']

In [18]:
len(words[0])

35

Данные готовы для применения кодов Хэмминга

In [19]:
def hamming_decode(words, size, n):
    shift = n - size
#Считаем матрицу преобразования
    matrix = np.vectorize(np.binary_repr)(np.arange(1,n + 1), width=shift)
    zero_arr = np.zeros((n, shift))
    for i in range(n):
        zero_arr[i] = np.vectorize(int)(list(matrix[i]))[::-1]
        
    result = ""
    for word in words:
#Переводим слова в нужный вид для работы
        binary_word = np.vectorize(int)(list(word)).reshape(-1)
        binary_word = binary_word[::-1]
        
#Считаем бит, где произошла ошибка      
        mistake = (binary_word @ zero_arr).reshape(-1)
        mistake = mistake[::-1]
        s = str((mistake) % 2)
        s = re.sub("[^0-9]", "", s)
        s = int(s, 2)
#Исправляем ошибку
        binary_word = binary_word.reshape(-1)
        if s > 0:
            binary_word[s - 1] = (1 + binary_word[s - 1]) % 2
        for i in range(shift):
            binary_word[2 ** i - 1] = -1
#Переводим слова опять к начальном виду и складываем вместе
        binary_word = binary_word[binary_word >= 0]
        binary_word = re.sub("[^0-9]", "", str(binary_word))
        result += binary_word
    return result

In [20]:
decoded_message = hamming_decode(words, size, n)

In [21]:
UTF8_SIZE = 8

Разбиваем на октеты, добавляем нули, переводим в чары

In [22]:
shift2 = UTF8_SIZE - len(decoded_message) % UTF8_SIZE

In [23]:
decoded_message = decoded_message + '0' * shift2

In [24]:
decoded_chars = re.findall('.{%s}' % UTF8_SIZE, decoded_message)

In [25]:
decoded_chars

['01001101',
 '01000101',
 '01010000',
 '01101000',
 '01001001',
 '00111110',
 '00100000',
 '00111000',
 '00110111',
 '00110100',
 '00110000',
 '00110111',
 '00110101',
 '00110001',
 '00100000',
 '00101010',
 '00100000',
 '00110110',
 '00110100',
 '00110110',
 '00110010',
 '00110111',
 '00110101',
 '00110110',
 '00100000',
 '00111101',
 '00100000',
 '00111111',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000']

In [26]:
char_ids = np.vectorize(int)(decoded_chars, base=2)

In [27]:
char_ids

array([ 77,  69,  80, 104,  73,  62,  32,  56,  55,  52,  48,  55,  53,
        49,  32,  42,  32,  54,  52,  54,  50,  55,  53,  54,  32,  61,
        32,  63,   0,   0,   0,   0,   0])

In [28]:
chars = np.vectorize(chr)(char_ids)

In [29]:
str(chars)

"['M' 'E' 'P' 'h' 'I' '>' ' ' '8' '7' '4' '0' '7' '5' '1' ' ' '*' ' ' '6'\n '4' '6' '2' '7' '5' '6' ' ' '=' ' ' '?' '' '' '' '' '']"

In [30]:
final_message = "".join(chars)

In [31]:
final_message

'MEPhI> 8740751 * 6462756 = ?'

In [32]:
answer = 8740751 * 6462756

In [33]:
answer

56489340969756

Теперь займемся енкодингом

Абсолютно аналогичная операция декодингу

In [34]:
def hamming_encode(words, n):
    size = len(words[0])
    shift = n - size
    matrix = np.vectorize(np.binary_repr)(np.arange(1,n + 1), width=shift)
    zero_arr = np.zeros((n, shift))
    for i in range(n):
        zero_arr[i] = np.vectorize(int)(list(matrix[i]))[::-1]
    result = ""
    for word in words:
        binary_word = np.vectorize(int)(list(word)).reshape(-1)
        for i in range(shift):
            binary_word = np.insert(binary_word, 2 ** i - 1, 0)
        mistake = (binary_word @ zero_arr).reshape(-1) % 2
        for i in range(shift):
            binary_word[2 ** i - 1] = mistake[i]
        binary_word = binary_word[::-1]
        binary_word = np.vectorize(str)(binary_word)
        binary_word = "".join(binary_word)
        result += binary_word
    return result

Приведем данные к нужному виду, переводим в биты, докидываем нулей

In [35]:
octets = np.vectorize(ord)(list(str(answer)))

In [36]:
octets

array([53, 54, 52, 56, 57, 51, 52, 48, 57, 54, 57, 55, 53, 54])

In [37]:
binary_answer = np.vectorize(np.binary_repr)(octets, width=8)

In [38]:
binary_answer

array(['00110101', '00110110', '00110100', '00111000', '00111001',
       '00110011', '00110100', '00110000', '00111001', '00110110',
       '00111001', '00110111', '00110101', '00110110'], dtype='<U8')

In [39]:
binary_answer = ''.join(binary_answer)

In [40]:
binary_answer

'0011010100110110001101000011100000111001001100110011010000110000001110010011011000111001001101110011010100110110'

In [41]:
shift3 = size - len(binary_answer) % size

In [42]:
shift3

4

In [43]:
binary_answer = binary_answer + '0' * shift3

In [44]:
len(binary_answer)

116

In [45]:
words_to_encode = re.findall('.{%s}' % size, binary_answer)

In [46]:
words_to_encode

['00110101001101100011010000111',
 '00000111001001100110011010000',
 '11000000111001001101100011100',
 '10011011100110101001101100000']

In [47]:
encoded_message = hamming_encode(words_to_encode, n)

In [48]:
shift4 = UTF8_SIZE - len(encoded_message) % UTF8_SIZE

In [49]:
encoded_message = encoded_message + '0' * shift4

In [50]:
encoded_message = re.findall('.{%s}' % UTF8_SIZE, encoded_message)

In [51]:
encoded_message

['11110000',
 '10110001',
 '10101001',
 '01001101',
 '00000000',
 '10110011',
 '00110011',
 '00111000',
 '00001100',
 '11110001',
 '10110010',
 '01111000',
 '01001011',
 '10000001',
 '10110010',
 '10110001',
 '11010100',
 '11000000']

In [52]:
encoded_message = np.vectorize(int)(encoded_message, base=2)

In [53]:
encoded_message

array([240, 177, 169,  77,   0, 179,  51,  56,  12, 241, 178, 120,  75,
       129, 178, 177, 212, 192])

In [54]:
encoded_message_str = np.vectorize(str)(encoded_message)

In [55]:
', '.join(encoded_message_str)

'240, 177, 169, 77, 0, 179, 51, 56, 12, 241, 178, 120, 75, 129, 178, 177, 212, 192'

Итоговое сообщение:

In [56]:
encoded_message

array([240, 177, 169,  77,   0, 179,  51,  56,  12, 241, 178, 120,  75,
       129, 178, 177, 212, 192])